<a href="https://colab.research.google.com/github/restrepo/OpenAlex/blob/main/json_total.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd
from google.colab import files
#From https://gist.github.com/korakot/072f1ea70d3c3267ff2c64b61d78c89c
import json
from IPython.display import HTML

def JSON(jstr):
  if type(jstr) != str:
    jstr = json.dumps(jstr)
  return HTML("""
<script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
<script>
renderjson.set_show_to_level(1)
document.body.appendChild(renderjson(%s))
new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
</script>
""" % jstr)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
with open('/content/drive/MyDrive/json/openalexco.json','r') as miarchivo:
    objeto=miarchivo.read()
objeto=json.loads(objeto)

Filter works with ror:

https://ror.org/03bp5hc83

In [8]:
objeto=[ c for c in objeto if any(
    [ any([i.get('ror')=="https://ror.org/03bp5hc83" for i in a.get('institutions')]) 
    for a in c.get('authorships')])
]
len(objeto)

34291

Define the filter which returns `True or False`

In [9]:
def filter_ror(authorships,ror="https://ror.org/03bp5hc83"):
  return any([any([i.get('ror')=="https://ror.org/03bp5hc83" for i in a.get('institutions')])  for a in authorships])

In [10]:
len([ c for c in objeto if filter_ror(c.get('authorships'))])

34291

Filter with Pandas

In [11]:
objeto=pd.DataFrame(objeto)

In [12]:
objeto[objeto['authorships'].apply(filter_ror)].shape

(34291, 25)

Explore concept levels from Excel file

In [13]:
cp=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vT9JrvUwanM-z-hCZ1SSaxRx2wfzLV3RVvm5BJaf8TABBmpIwKt7iYt1sVLbImVevh9YNaZK9Y7sfI9/pub?output=xlsx')

Level 1 Physics concepts

In [14]:
ph1=cp[ (cp['level']==1) & (cp['parent_display_names'].fillna('').str.contains('Physics')) 
      ]['display_name'].to_list()

In [15]:
len(ph1)

27

Level 2 Physics concepts

In [16]:
ph2=cp[(cp['level']==2) & (cp['parent_display_names'].str.split(', ').apply(
    lambda l: set(l).intersection(set(ph1)) if isinstance(l,list) else set()
    ).apply(len)>0)]['display_name'].to_list()
len(ph2)

2979

Level 0 UdeA Physics articles

In [17]:
possible_physics=objeto[objeto['concepts'].apply(lambda l: {'Physics'}.intersection( 
    [d.get('display_name') for d in l if d.get('level')==0])).apply(len)>0]
possible_physics.shape


(1886, 25)

Level 1 UdeA Physics concepts

In [18]:
ll=possible_physics['concepts'].apply(lambda l: set([d.get('display_name') for d in l if d.get('level')==1]).intersection(ph1)).apply(list)
ph1_udea=set([item for subitem in ll for item in subitem])

Missing Level 1 concepts

In [19]:
set(ph1).difference(ph1_udea)

{'Geophysics', 'Medical physics'}

Level 2 UdeA Physics concepts

In [123]:
ll=possible_physics['concepts'].apply(lambda l: set([d.get('display_name') for d in l if d.get('level')==2]).intersection(ph2)).apply(list)
ph2_udea=set([item for subitem in ll for item in subitem])

In [185]:
len(ph2_udea)

507

UdeA Physics concepts tree

In [186]:
d={'Physics': dict( [(c,[]) for c in ph1_udea])}
for c in ph2_udea:
    lph1=set(ph1_udea).intersection( cp[cp['display_name']==c]['parent_display_names'].str.split(', ').iloc[0])
    for cc in lph1:
        if cc in d['Physics'] and c not in d['Physics'][cc]:
            d['Physics'][cc].append(c)


In [187]:
JSON(d)

In [ ]:

f=open('openalexua.json','w')
json.dump(objeto,f)
f.close()

In [50]:

x=set()
type(x)

set

In [ ]:
f=open('openalexua.json','r')
df=json.load(f)
f.close()